In [81]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [71]:
nwsRequest = requests.get('https://forecast.weather.gov/MapClick.php?lat=40.26289286900004&lon=-76.88102168999995#.X8_bXGRKhhF')
nwsRequest

<Response [200]>

In [72]:
nws = BeautifulSoup(nwsRequest.content, 'html.parser')

#Find the div with id seven-day-forcast, assign to a variable seven-day
sevenDay = nws.find(id='seven-day-forecast')

#inside sevenDay, find each individual forecast item
forcastItems = sevenDay.find_all('div', attrs={'class':'tombstone-container'})
# forcast_items = seven_day.select("[class~='tombstone-container']")
tonight = forcastItems[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Increasing clouds, with a low around 29. West wind 3 to 8 mph. Winds could gust as high as 20 mph. " class="forecast-icon" src="newimages/medium/nsct.png" title="Tonight: Increasing clouds, with a low around 29. West wind 3 to 8 mph. Winds could gust as high as 20 mph. "/>
 </p>
 <p class="short-desc">
  Increasing
  <br/>
  Clouds
 </p>
 <p class="temp temp-low">
  Low: 29 °F
 </p>
</div>


In [73]:
#extract a single-period of information from the page - 4 pices

#name of the forcast period
period = tonight.find(class_="period-name").get_text()

#description of the conditions - its stored in teh title property of the img tag
img= tonight.find("img")
# print(img.prettify()) -> this statement used to print output cleanly
desc= img["title"]
#Short description
shortDesc = tonight.find(class_='short-desc').get_text()
#the temparature 
temp = tonight.find(class_="temp").get_text()

In [74]:
print(period)
print(desc)
print(shortDesc)
print(temp)

Tonight
Tonight: Increasing clouds, with a low around 29. West wind 3 to 8 mph. Winds could gust as high as 20 mph. 
IncreasingClouds
Low: 29 °F


In [75]:
#extract all info form section of the page

#select all teh items with the class period-name inside an item with the 
#class tomdstone-container in seven_Day

periodTags = sevenDay.select(".tombstone-container .period-name")
periodTags

#User a list comprehension to call the get_text method on each BeautifuSoup object
periods = [pt.get_text() for pt in periodTags]
periods

['Tonight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight']

In [82]:
#Get the rest the information
shortDescs = [sd.get_text() for sd in sevenDay.select(".tombstone-container .short-desc")]
temps = [tp.get_text() for tp in sevenDay.select(".tombstone-container .temp")]
descs = [d['title'] for d in sevenDay.select(".tombstone-container img")]

In [83]:
print(shortDescs)
print(temps)
print(descs)

['IncreasingClouds', 'Slight ChanceLight Snowthen MostlyCloudy', 'Mostly Cloudy', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Mostly Cloudy', 'Mostly Cloudy', 'Mostly Cloudythen ChanceRain']
['Low: 29 °F', 'High: 41 °F', 'Low: 33 °F', 'High: 49 °F', 'Low: 30 °F', 'High: 53 °F', 'Low: 37 °F', 'High: 55 °F', 'Low: 44 °F']
['Tonight: Increasing clouds, with a low around 29. West wind 3 to 8 mph. Winds could gust as high as 20 mph. ', 'Wednesday: A slight chance of light snow before 11am.  Mostly cloudy, with a high near 41. West wind 3 to 8 mph.  Chance of precipitation is 20%.', 'Wednesday Night: Mostly cloudy, with a low around 33. West wind around 6 mph. ', 'Thursday: Sunny, with a high near 49. Northwest wind 3 to 6 mph. ', 'Thursday Night: Mostly clear, with a low around 30. Calm wind. ', 'Friday: Mostly sunny, with a high near 53.', 'Friday Night: Mostly cloudy, with a low around 37.', 'Saturday: Mostly cloudy, with a high near 55.', 'Saturday Night: A chance of rain after 1am.  Mostl

In [86]:
# Combine into a dataframe to analyse
weather = pd.DataFrame({
    "period":periods,
    "shortDesc":shortDescs,
    "temp":temps,
    "desc":descs
})
weather
#use a dictionary to creat dataframe

,period,shortDesc,temp,desc
0,Tonight,IncreasingClouds,Low: 29 °F,"Tonight: Increasing clouds, with a low around ..."
1,Wednesday,Slight ChanceLight Snowthen MostlyCloudy,High: 41 °F,Wednesday: A slight chance of light snow befor...
2,WednesdayNight,Mostly Cloudy,Low: 33 °F,"Wednesday Night: Mostly cloudy, with a low aro..."
3,Thursday,Sunny,High: 49 °F,"Thursday: Sunny, with a high near 49. Northwes..."
4,ThursdayNight,Mostly Clear,Low: 30 °F,"Thursday Night: Mostly clear, with a low aroun..."
5,Friday,Mostly Sunny,High: 53 °F,"Friday: Mostly sunny, with a high near 53."
6,FridayNight,Mostly Cloudy,Low: 37 °F,"Friday Night: Mostly cloudy, with a low around..."
7,Saturday,Mostly Cloudy,High: 55 °F,"Saturday: Mostly cloudy, with a high near 55."
8,SaturdayNight,Mostly Cloudythen ChanceRain,Low: 44 °F,Saturday Night: A chance of rain after 1am. M...


In [101]:
#pull out numeric temp values using a regular expression and series class
tempNums = weather["temp"].str.extract("(?P<tempNum> \d+)", expand=False)
tempNums

#add a new coloumn to a data frame
weather["tempNum"] = tempNums.astype('int')

In [102]:
weather

,period,shortDesc,temp,desc,tempNum
0,Tonight,IncreasingClouds,Low: 29 °F,"Tonight: Increasing clouds, with a low around ...",29
1,Wednesday,Slight ChanceLight Snowthen MostlyCloudy,High: 41 °F,Wednesday: A slight chance of light snow befor...,41
2,WednesdayNight,Mostly Cloudy,Low: 33 °F,"Wednesday Night: Mostly cloudy, with a low aro...",33
3,Thursday,Sunny,High: 49 °F,"Thursday: Sunny, with a high near 49. Northwes...",49
4,ThursdayNight,Mostly Clear,Low: 30 °F,"Thursday Night: Mostly clear, with a low aroun...",30
5,Friday,Mostly Sunny,High: 53 °F,"Friday: Mostly sunny, with a high near 53.",53
6,FridayNight,Mostly Cloudy,Low: 37 °F,"Friday Night: Mostly cloudy, with a low around...",37
7,Saturday,Mostly Cloudy,High: 55 °F,"Saturday: Mostly cloudy, with a high near 55.",55
8,SaturdayNight,Mostly Cloudythen ChanceRain,Low: 44 °F,Saturday Night: A chance of rain after 1am. M...,44


In [104]:
type(weather['tempNum'][0])

numpy.int64

In [106]:
#we can find the mean of all the temps for the week
weather["tempNum"].mean()

41.22222222222222

In [108]:
#we can select only the rows that happen at night
isNight = weather["temp"].str.contains("Low")
isNight
weather["isNight"] = isNight

In [109]:
weather

,period,shortDesc,temp,desc,tempNum,isNight
0,Tonight,IncreasingClouds,Low: 29 °F,"Tonight: Increasing clouds, with a low around ...",29,True
1,Wednesday,Slight ChanceLight Snowthen MostlyCloudy,High: 41 °F,Wednesday: A slight chance of light snow befor...,41,False
2,WednesdayNight,Mostly Cloudy,Low: 33 °F,"Wednesday Night: Mostly cloudy, with a low aro...",33,True
3,Thursday,Sunny,High: 49 °F,"Thursday: Sunny, with a high near 49. Northwes...",49,False
4,ThursdayNight,Mostly Clear,Low: 30 °F,"Thursday Night: Mostly clear, with a low aroun...",30,True
5,Friday,Mostly Sunny,High: 53 °F,"Friday: Mostly sunny, with a high near 53.",53,False
6,FridayNight,Mostly Cloudy,Low: 37 °F,"Friday Night: Mostly cloudy, with a low around...",37,True
7,Saturday,Mostly Cloudy,High: 55 °F,"Saturday: Mostly cloudy, with a high near 55.",55,False
8,SaturdayNight,Mostly Cloudythen ChanceRain,Low: 44 °F,Saturday Night: A chance of rain after 1am. M...,44,True
